In [3]:
#load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy 
import PIL
import requests

%pip install plotly
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [9]:
ttcdelay = pd.read_excel("/Users/gregantono/Desktop/TTC-2024delays.xlsx", engine='openpyxl')

ttcdelay.columns = ttcdelay.columns.str.strip().str.lower()

# Exclude 'Other' causes 
ttcdelay = ttcdelay[ttcdelay['cause'].str.lower() != 'other']

# Check sample
print(ttcdelay.head())

                                               cause   code line
0  Passenger Assistance Alarm Activated - No Trou...  MUPAA   YU
2  Passenger Assistance Alarm Activated - No Trou...  MUPAA   YU
3  Passenger Assistance Alarm Activated - No Trou...  MUPAA   YU
4                   Door Problems - Faulty Equipment   EUDO   BD
5   Injured or ill Customer (On Train) - Transported    MUI   YU


In [21]:
cause_counts = ttcdelay.groupby(['code', 'cause']).size().reset_index(name='count')
top20_causes = cause_counts.nlargest(20, 'count').copy()

In [28]:
hover_text = [
    f"Cause: {cause}<br>Count: {count}"
    for cause, count in zip(top20_causes['cause'], top20_causes['count'])
]

graph = go.Figure()
graph.add_trace(go.Bar(x=top20_causes['code'], 
                       y=top20_causes['count'], 
                       text=hover_text, 
                       textposition='none'                       ))

graph.update_layout(
    title = 'Top 20 causes of TTC delays in 2024',
    xaxis_title = 'Cause of delay',
    yaxis_title = 'Frequency',
    xaxis_tickangle=65
)

In [29]:
#define a subset that consists of passenger-related causes

passenger_causes = {'SUDP', 'MUPAA', 'SUO', 'MUIR', 'MUSAN','SUUT', 'MUI', 'SUAP', 'SUG'}

# redefining colors for subset

colors = ['indianred' if code in passenger_causes else 'steelblue'
          for code in top20_causes['code']]

#recreating plot integrating this subset

graph = go.Figure()
graph.add_trace(go.Bar(x=top20_causes['code'], 
                       y=top20_causes['count'], 
                       text=hover_text, 
                       textposition='none',
                                              marker_color=colors                       ))

graph.update_layout(
    title = 'Top 20 causes of TTC delays in 2024',
    xaxis_title = 'Cause of delay',
    yaxis_title = 'Frequency',
    xaxis_tickangle=65
)


Attempting a different version of this graph, in order to further differentiate the bars according to the subway line.

In [45]:
df = pd.read_excel("/Users/gregantono/Desktop/TTC-2024delays.xlsx", engine='openpyxl')

df.columns = df.columns.str.strip().str.lower()
df = df[df['cause'].str.lower() != 'other']  #excluding those labelled 'other'

grouped = df.groupby(['code', 'cause', 'line']).size().reset_index(name='count')

pivot = grouped.pivot_table(index=['code', 'cause'], columns='line', values='count', fill_value=0)
pivot = pivot.reset_index()

pivot['total'] = pivot.iloc[:, 2:].sum(axis=1)   #sorting by total count
top_20 = pivot.sort_values(by='total', ascending=False).head(20).copy() #extract top 20 


fig = go.Figure()      

line_colors = {                     
    'YU': '#EECA3B',                    # customize Line color and full name mapping
    'BD': 'seagreen',
    'SHP': 'indianred'
}

line_name_map = {                       #to display full line name instead of the TTC subway code
    'YU': 'Yonge-University',
    'BD': 'Bloor-Danforth',
    'SHP': 'Sheppard-Yonge'
}

line_totals = top_20.iloc[:, 2:-1].sum().sort_values(ascending=False)
stack_order = line_totals.index.tolist()                                # Order TTC lines in order of contribution

for line_code in stack_order:
    full_name = line_name_map.get(line_code, line_code)
    fig.add_trace(
        go.Bar(
            x=top_20['code'],
            y=top_20[line_code],
            name=full_name,  # Show full name in legend
            marker_color=line_colors.get(line_code, 'gray'),
            hovertext=[
                f"Cause: {cause}<br>Line: {full_name}<br>Count: {count}"
                for cause, count in zip(top_20['cause'], top_20[line_code])
            ],
            hoverinfo='text'
        )
    )


fig.add_trace(                          #To display the total counts for each cause of delay above each bar
    go.Scatter(
        x=top_20['code'],
        y=top_20['total'],
        mode='text',
        text=top_20['total'],
        textposition='top center',
        textfont=dict(size=10),
        showlegend=False,
        hoverinfo='skip'
    )
)

fig.update_layout(
    title="Top 20 TTC Delay Causes in 2024",
    xaxis_title="Cause Code",
    yaxis_title="Frequency",
    xaxis_tickangle=65,
    barmode='stack',
    height=600,
    margin=dict(t=80, b=120)
)


fig.show()
